# MC_simulations

## This script does the following

* Computes full-order snapshots for the Monte Carlo simulation

In [ ]:
start = time.time()

class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0.0)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 1.0)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 1.0)

# Define subdomains 

class Block0(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (2/3, 1)) and between(x[0], (0, 1/3)))
    
class Block1(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (2/3, 1)) and between(x[0], (1/3, 2/3)))

class Block2(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (2/3, 1)) and between(x[0], (2/3, 1)))
    
class Block3(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (1/3, 2/3)) and between(x[0], (0, 1/3)))
    
class Block4(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (1/3, 2/3)) and between(x[0], (1/3, 2/3)))
    
class Block5(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (1/3, 2/3)) and between(x[0], (2/3, 1)))
    
class Block6(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0, 1/3)) and between(x[0], (0, 1/3)))
    
class Block7(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0, 1/3)) and between(x[0], (1/3, 2/3)))

class Block8(SubDomain):
    def inside(self, x, on_boundary):
        return (between(x[1], (0, 1/3)) and between(x[0], (2/3, 1))) 
    
# Initialize sub-domain instances
left = Left()
top = Top()

right = Right()
bottom = Bottom()

block0 = Block0()
block1 = Block1()
block2 = Block2()

block3 = Block3()
block4 = Block4()
block5 = Block5()

block6 = Block6()
block7 = Block7()
block8 = Block8()

# Define mesh
gridsize = 99
mesh = UnitSquareMesh(gridsize, gridsize)

# Define function space and basis functions
V = FunctionSpace(mesh, "Lagrange", 1)
u = Function(V)
v = TestFunction(V)

# Initialize mesh function for interior domains
domains = MeshFunction("size_t", mesh,2)
domains.set_all(0)

block0.mark(domains, 0)
block1.mark(domains, 1)
block2.mark(domains, 2)

block3.mark(domains, 3)
block4.mark(domains, 4)
block5.mark(domains, 5)

block6.mark(domains, 6)
block7.mark(domains, 7)
block8.mark(domains, 8)

# Initialize mesh function for boundary domains
boundaries = MeshFunction("size_t", mesh,1)
boundaries.set_all(0)

left.mark(boundaries, 1)
right.mark(boundaries,2)

top.mark(boundaries, 3)
bottom.mark(boundaries, 4)

# Get location of dofs
dofs_x = V.tabulate_dof_coordinates().reshape((-1, 2))

# Select indices that correspond to the dofs on domain 1, 2 and interface respectively

part1_rows_domain1 = np.where(( dofs_x[:,1]>2/3) | (dofs_x[:,1]<1/3 ))[0]
part2_rows_domain1 = np.where(( dofs_x[:,0]<1/3) & (dofs_x[:,1]>=1/3 ) & (dofs_x[:,1]<=2/3) )[0]
part3_rows_domain1 = np.where(( dofs_x[:,0]>2/3) & (dofs_x[:,1]>=1/3 ) & (dofs_x[:,1]<=2/3) )[0]

rows_domain1 = np.hstack([part1_rows_domain1,part2_rows_domain1,part3_rows_domain1])
rows_domain2 = np.where( (dofs_x[:,1]<2/3) & (dofs_x[:,1]>1/3) & (dofs_x[:,0]>1/3) & (dofs_x[:,0]<2/3) )[0]

rows_interface = []
rows_domain12 = np.hstack([rows_domain1,rows_domain2])

for i in range(0,len(dofs_x)):
    if not i in rows_domain12:
        rows_interface.append(i)

E_1 = sps.csc_matrix((np.ones(len(rows_domain1)),(rows_domain1,np.arange(len(rows_domain1)))), shape=(V.dim(),len(rows_domain1)))
E_2 = sps.csc_matrix((np.ones(len(rows_domain2)),(rows_domain2,np.arange(len(rows_domain2)))), shape=(V.dim(),len(rows_domain2)))
E_interface = sps.csc_matrix((np.ones(len(rows_interface)),(rows_interface,np.arange(len(rows_interface)))), shape=(V.dim(),len(rows_interface)))
        
# Define input data
g_T = Expression("4*x[1]",degree=1)
g_B = Expression("4*x[1]",degree=1)

f0 = Constant(-6)
f1 = Constant(-6)
f2 = Constant(-6)
f3 = Constant(-6)

f4 = Constant(-6)
f5 = Constant(-6)
f6 = Constant(-6)

f7 = Constant(-6)
f8 = Constant(-6)

# Set seed for reproducability
random.seed(102343)

# Define parameters

para0 = [0.1, 1]
para1 = [0.1, 1]
para2 = [0.1, 1]

para3 = [0.1, 1]
para4 = [0.1, 1]
para5 = [0.1, 1]

para6 = [0.1, 1]
para7 = [0.1, 1]
para8 = [0.1, 1]

# Define the number of parameters, number of test samples, number of snapshots
params = 9 
# ntest = 1
# nsamples = 100 + ntest

# define size of the space
nh = (gridsize+1)**2 

# Initialize mu and S
mu = np.zeros((nsamples,params))
S = np.zeros((nh,nsamples))

# Create arrays for vectors F
ndofs_U1 = len(rows_domain1)
ndofs_UI = len(rows_interface)
ndofs_U2 = len(rows_domain2)

S1 = np.zeros((ndofs_U1,nsamples))
SI = np.zeros((ndofs_UI,nsamples))
S2 = np.zeros((ndofs_U2,nsamples))

# Create arrays for matrices A_** and vectors F_*

A_11_array = []
A_1I_array = []

A_I1_array = []
A_II1_array = []

F_1_array = np.zeros((nsamples,ndofs_U1)) 
F_1I_array = np.zeros((nsamples,ndofs_UI))

# Define new measures associated with the interior domains and
# exterior boundaries
dx = Measure("dx")(subdomain_data = domains)
ds = Measure("ds")(subdomain_data = boundaries)

# Define Dirichlet boundary conditions at top and bottom boundaries
u0 = Expression("1+x[0]*x[0]+2*x[1]*x[1]", degree=2)

# Define dirichlet boundary conditions
bcs = [DirichletBC(V, u0, boundaries, 1),
DirichletBC(V, u0, boundaries, 2)]

count = 0 

end= time.time()
snapshots_timee = end - start

m3_time = 0

for i in range(nsamples):
    
    start = time.time()
    
    par0r0, par1r0, par2r0, par3r0, par4r0, par5r0, par6r0, par7r0, par8r0 = random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), random.uniform(0,1)
    
    par0 = (para0[1]-para0[0])*par0r0+para0[0]
    par1 = (para1[1]-para1[0])*par1r0+para1[0]
    par2 = (para2[1]-para2[0])*par2r0+para2[0]

    par3 = (para3[1]-para3[0])*par3r0+para3[0]
    par4 = (para4[1]-para4[0])*par4r0+para4[0]
    par5 = (para5[1]-para5[0])*par5r0+para5[0]

    par6 = (para6[1]-para6[0])*par6r0+para6[0]
    par7 = (para7[1]-para7[0])*par7r0+para7[0]
    par8 = (para8[1]-para8[0])*par8r0+para8[0]
    
#     par0, par1, par2, par3, par4 = .5, .5, .5, .5, .5
#     par5, par6, par7, par8 = 0.5, .5, .5, .5
        
    if i == nsamples - ntest:
        
        snapshots_time = snapshots_timee
    
    # Store the parameters
    mu[i] = [par0, par1, par2, par3, par4, par5, par6, par7, par8]
    
    u = Function(V)
    g_u = 1+u
    
    F = (
        
        inner(par0*grad(u), grad(v))*dx(0) + 
        inner(par1*grad(u), grad(v))*dx(1) + 
        inner(par2*grad(u), grad(v))*dx(2) +
        
        inner(par3*grad(u), grad(v))*dx(3) + 
        inner(par4*g_u*grad(u), grad(v))*dx(4) +
        inner(par5*grad(u), grad(v))*dx(5) + 
        
        inner(par6*grad(u), grad(v))*dx(6) + 
        inner(par7*grad(u), grad(v))*dx(7) + 
        inner(par8*grad(u), grad(v))*dx(8)  
        
        - f0*v*dx(0) - f1*v*dx(1) - f2*v*dx(2) 
        - f3*v*dx(3) - f4*v*dx(4) - f5*v*dx(5)
        - f6*v*dx(6) - f7*v*dx(7) - f8*v*dx(8) 
        
        - g_T*v*ds(3) - g_B*v*ds(4) 
        )
    
    # Compute solution
    solve(F == 0, u, bcs,
          solver_parameters={"newton_solver":{"relative_tolerance":1e-1}})  

    nodal_values = u.vector().get_local()
        
    S1[:,i] = nodal_values[rows_domain1]
    S2[:,i] = nodal_values[rows_domain2]
    SI[:,i] = nodal_values[rows_interface]
    
    S[0:ndofs_U1,i] = S1[:,i]
    S[ndofs_U1:ndofs_U1+ndofs_U2,i] = S2[:,i] 
    S[-ndofs_UI:,i] = SI[:,i]

    
    end = time.time()
    
    iter_time = end - start
    snapshots_timee += iter_time
    
    # ------------ Linear part ----------------------------
    
    start = time.time()
    
    u = TrialFunction(V)
    
    F1 = (
        inner(par0*grad(u), grad(v))*dx(0) + 
        inner(par1*grad(u), grad(v))*dx(1) + 
        inner(par2*grad(u), grad(v))*dx(2) + 
        
        inner(par3*grad(u), grad(v))*dx(3) + 
        inner(par5*grad(u), grad(v))*dx(5) + 
        inner(par6*grad(u), grad(v))*dx(6) + 
        
        inner(par7*grad(u), grad(v))*dx(7) + 
        inner(par8*grad(u), grad(v))*dx(8)  
        - f0*v*dx(0) - f1*v*dx(1) - f2*v*dx(2) 
        
        - f3*v*dx(3) -  f5*v*dx(5)
        - f6*v*dx(6) - f7*v*dx(7) - f8*v*dx(8) 
        - g_T*v*ds(3) - g_B*v*ds(4) 
        )
    
    a_1, L1 = lhs(F1), rhs(F1)
    
    # Transform UFL object to a sparse csc
    
    a1, f_1 = UFL_to_csc(a_1,L1,bcs)

    # Create block matrices
    A_11 = E_1.T*a1*E_1
    A_II1 = E_interface.T*a1*E_interface

    A_1I = E_1.T*a1*E_interface
    A_I1 = E_interface.T*a1*E_1
    
    F_1 = f_1[rows_domain1]
    F_1I = f_1[rows_interface]
    
    A_11_array.append(A_11) 
    A_1I_array.append(A_1I)
    
    A_I1_array.append(A_I1)
    A_II1_array.append(A_II1)
    
    F_1_array[i] = F_1
    F_1I_array[i] = F_1I
    
    end=time.time()
    
    iter_m3 = end-start
    
    m3_time += iter_m3
    
    if i == nsamples - ntest:
        m3_time_train = m3_time
    
    print(i+1, "snapshots computed,",nsamples-i-1,"snapshots to go...", end="\r")

print("\n \n Finished")
# m3_time_onl = m3_time - m3_time_train
